In [12]:
import sys
import os
from pathlib import Path
sys.path.append(Path.cwd().parent.as_posix())
from util import *
from visualizer import visualizer
vis = visualizer()

from mattergen.self_guidance.wyckoff_dataset_prim import *
from mattergen.self_guidance.wyckoff_sampler_prim_new import *
from mattergen.common.data.collate import collate

In [13]:
# def get_latttice_permutations(device="cpu"):
#     lat_perm = np.zeros((231, 3, 3, 3, 3, 3))
#     lat_perm[:, :, :, :, 0, 0] = 1
#     lat_perm[:, :, :, :, 1, 1] = 1
#     lat_perm[:, :, :, :, 2, 2] = 1
#     # Ex) 1, 0, 2 implies lat(1) > lat(0) > lat(2)
#     possible_perms = [[0, 1, 2], [0, 2, 1], [1, 0, 2], [1, 2, 0], [2, 0, 1], [2, 1, 0]]

#     perm_for_A1 = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
#     perm_for_A2 = np.eye(3).reshape(1, 3, 3).repeat(231, axis=0)
#     lat_perm[:, 0, 1, 2] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
#     lat_perm[:, 0, 2, 1] = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
#     lat_perm[:, 1, 0, 2] = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
#     lat_perm[:, 1, 2, 0] = np.array([[1, 0, 0], [0, 0, 1], [0, 1, 0]])
#     lat_perm[:, 2, 0, 1] = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]])
#     lat_perm[:, 2, 1, 0] = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

#     # orthorhombic with A:
#     for i in [38, 39, 40, 41]:
#         for p in possible_perms:
#             lat_perm[i, p[0], p[1], p[2]] = perm_for_A1 @ lat_perm[0, p[0], p[1], p[2]]
#         # perm_for_A2[i] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
#     lat_perm = torch.tensor(lat_perm, dtype=torch.float32)
#     perm_for_A1 = torch.tensor(perm_for_A1, dtype=torch.float32)
#     perm_for_A2 = torch.tensor(perm_for_A2, dtype=torch.float32)
#     return lat_perm.to(device), perm_for_A1.to(device), perm_for_A2.to(device)

# def _project_to_space_group(batch,pos=None,cell=None):
#     cf = CrystalFamily()
#     cf.set_device(batch.pos.device)
#     lat_perm, _, perm_for_A2 = get_latttice_permutations(device=batch.pos.device)
#     if pos is not None:
#         batch = batch.replace(pos=pos)
#     else:
#         batch = batch.replace(pos=torch.rand_like(batch.pos))
#     if cell is not None:
#         batch = batch.replace(cell=cell)
#     else:
#         batch = batch.replace(cell=torch.tensor(initialize_random_lattice().matrix).unsqueeze(0).float())

#     wyckoff_batch = batch.wyckoff_bat.clone()
#     idx, cum = 0, 0
#     for len, num_atom in zip(batch.wyckoff_bat_len, batch.num_atoms):
#         wyckoff_batch[idx : idx + len] = wyckoff_batch[idx : idx + len] + cum
#         idx += len
#         cum += num_atom

#     # project latice
#     conv_lat = torch.bmm(batch.prim_to_conv, batch.cell)
#     conv_lat_vec = cf.m2v(cf.de_so3(conv_lat))
#     conv_lat_vec_proj = cf.proj_k_to_spacegroup(conv_lat_vec, batch.space_groups)
#     conv_lat_proj = cf.v2m(conv_lat_vec_proj)
#     print("conv_lat_proj") # The same
#     print(conv_lat_proj)

#     rank = torch.argsort(-torch.norm(conv_lat_proj, dim=-1), dim=-1)
#     idx = torch.cat([batch.space_groups.unsqueeze(-1), rank], dim=-1)
#     perm = lat_perm[idx[:, 0], idx[:, 1], idx[:, 2], idx[:, 3]]
#     perm_A = perm_for_A2[batch.space_groups]
#     # perm = perm_for_A2[0].unsqueeze(0).repeat(batch.space_groups.shape[0], 1, 1)
    

#     perm_conv_lat_proj = torch.bmm(torch.bmm(perm, conv_lat_proj), perm.transpose(-1, -2))
#     perm_conv_lat_proj = torch.bmm(
#         torch.bmm(perm_A, perm_conv_lat_proj), perm_A.transpose(-1, -2)
#     )
#     # perm_conv_lat_proj = conv_lat_proj

#     prim_lat_proj = torch.bmm(torch.bmm(torch.bmm(perm_A,batch.conv_to_prim),perm_A.transpose(-1,-2)), perm_conv_lat_proj)
#     prim_lat_proj = torch.bmm(batch.conv_to_prim, perm_conv_lat_proj)
#     print(perm)
#     print(perm.transpose(-1,-2))
#     print(perm_A)
#     print(perm_conv_lat_proj)

#     batch_cell = torch.bmm(torch.bmm(perm, batch.cell), perm.transpose(-1, -2)) 
#     pos_cart = torch.einsum("bi,bij->bj", batch.pos, batch_cell[batch.batch])
#     print("pos_cart")
#     print(pos_cart)
#     pos_frac_conv = torch.einsum(
#         "bi,bij->bj", pos_cart, torch.inverse(perm_conv_lat_proj)[batch.batch]
#     )
#     pos_frac_conv = pos_frac_conv % 1.0
#     print("pos_frac_conv")
#     print(pos_frac_conv)
#     pos_tran = torch.cat(
#         [
#             pos_frac_conv[wyckoff_batch],
#             torch.ones(pos_frac_conv[wyckoff_batch].shape[0], 1, device=batch.pos.device),
#         ],
#         dim=1,
#     )

#     pos_frac_proj = (
#         torch.einsum("bij,bj->bi", batch.wyckoff_ops, pos_tran).squeeze(-1)[:, :3] % 1.0
#     )
#     no_perm_pos_frac_proj = pos_frac_proj.clone()
#     pos_frac_proj = torch.einsum("bij,bj->bi", perm[batch.batch[wyckoff_batch]], pos_frac_proj)
#     pos_cart_proj = torch.einsum(
#         "bi,bij->bj", pos_frac_proj, perm_conv_lat_proj[batch.batch[wyckoff_batch]]
#     )
#     # print("pos_cart_proj")
#     # print(pos_cart_proj)

#     prim_lat_inv = torch.inverse(prim_lat_proj)
#     pos_prim_frac_proj_all = torch.einsum(
#         "bi,bij->bj", pos_cart_proj, prim_lat_inv[batch.batch[wyckoff_batch]]
#     )
#     # print("pos_prim_frac_proj_all")
#     # print(pos_prim_frac_proj_all)

#     ## Get prim idx
#     for i in range(10):
#         random_pos_frac_conv = torch.rand_like(pos_frac_conv).to(pos_frac_conv.device)
#         random_pos_tran = torch.cat(
#             [
#                 random_pos_frac_conv[wyckoff_batch],
#                 torch.ones(
#                     random_pos_frac_conv[wyckoff_batch].shape[0],
#                     1,
#                     device=pos_frac_conv.device,
#                 ),
#             ],
#             dim=1,
#         )
#         random_pos_frac_proj = (
#             torch.einsum("bij,bj->bi", batch.wyckoff_ops, random_pos_tran).squeeze(-1)[
#                 :, :3
#             ]
#             % 1.0
#         ) % 1.0
#         random_pos_frac_proj = torch.einsum(
#             "bij,bj->bi", perm[batch.batch[wyckoff_batch]], random_pos_frac_proj
#         )
#         random_pos_cart_proj = torch.einsum(
#             "bi,bij->bj",
#             random_pos_frac_proj,
#             perm_conv_lat_proj[batch.batch[wyckoff_batch]],
#         )

#         random_fracs = torch.einsum(
#             "bi,bij->bj",
#             random_pos_cart_proj,
#             prim_lat_inv[batch.batch[wyckoff_batch]],
#         )
#         random_fracs = random_fracs % 1.0
#         random_fracs_diff = random_fracs.unsqueeze(1) - random_fracs.unsqueeze(0)
#         random_fracs_diff = random_fracs_diff - torch.round(random_fracs_diff)
#         EPSILON = 5e-4
#         random_fracs_diff_is_zero = torch.all(
#             torch.isclose(
#                 random_fracs_diff,
#                 torch.zeros_like(random_fracs_diff),
#                 rtol=EPSILON,
#                 atol=EPSILON,
#             ),
#             dim=-1,
#         )
#         random_fracs_idx = random_fracs_diff_is_zero & (
#             wyckoff_batch.unsqueeze(0) == wyckoff_batch.unsqueeze(1)
#         )
#         random_fracs_idx = ~(random_fracs_idx.triu(diagonal=1).any(dim=0))
#         # random_fracs_prim = random_fracs[random_fracs_idx]
#         assert random_fracs_idx.shape[0] == pos_prim_frac_proj_all.shape[0]
#         pos_prim_frac_proj = pos_prim_frac_proj_all[random_fracs_idx]
#         if pos_prim_frac_proj.shape[0] == batch.pos.shape[0]:
#             frac_batch = random_fracs_idx
#             break
#     pos_prim_frac_proj = pos_prim_frac_proj_all[frac_batch]

#     # prim_lat_proj = torch.bmm(torch.bmm(perm.transpose(-1,-2), prim_lat_proj),perm)
#     # pos_prim_frac_proj = torch.einsum("bij,bj->bi", perm.transpose(-1,-2), pos_prim_frac_proj)

#     return (pos_prim_frac_proj % 1.0), prim_lat_proj, pos_frac_proj % 1.0, perm_conv_lat_proj
# # Success for 6

In [14]:
def get_latttice_permutations(device="cpu"):
    lat_perm = np.zeros((231, 3, 3, 3, 3, 3))
    lat_perm[:, :, :, :, 0, 0] = 1
    lat_perm[:, :, :, :, 1, 1] = 1
    lat_perm[:, :, :, :, 2, 2] = 1
    # Ex) 1, 0, 2 implies lat(1) > lat(0) > lat(2)
    possible_perms = [[0, 1, 2], [0, 2, 1], [1, 0, 2], [1, 2, 0], [2, 0, 1], [2, 1, 0]]

    perm_for_A1 = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
    perm_for_A2 = lat_perm.copy()
    # lat_perm[:, 0, 1, 2] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
    # lat_perm[:, 0, 2, 1] = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
    # lat_perm[:, 1, 0, 2] = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
    # lat_perm[:, 1, 2, 0] = np.array([[1, 0, 0], [0, 0, 1], [0, 1, 0]])
    # lat_perm[:, 2, 0, 1] = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]])
    # lat_perm[:, 2, 1, 0] = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

    # orthorhombic with A:
    for i in [38, 39, 40, 41]:
        # perm_for_A2[i, 0, 1, 2] = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
        # perm_for_A2[i, 0, 2, 1] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
        # perm_for_A2[i, 1, 0, 2] = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
        # perm_for_A2[i, 1, 2, 0] = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
        # perm_for_A2[i, 2, 0, 1] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
        # perm_for_A2[i, 2, 1, 0] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
        
        perm_for_A2[i, 0, 1, 2] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
        perm_for_A2[i, 0, 2, 1] = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
        perm_for_A2[i, 1, 0, 2] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
        perm_for_A2[i, 1, 2, 0] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
        perm_for_A2[i, 2, 0, 1] = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
        perm_for_A2[i, 2, 1, 0] = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])

        # perm_for_A2[i, 0, 1, 2] = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]])
        # perm_for_A2[i, 0, 2, 1] = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
        # perm_for_A2[i, 1, 0, 2] = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]])
        # perm_for_A2[i, 1, 2, 0] = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]])
        # perm_for_A2[i, 2, 0, 1] = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]])
        # perm_for_A2[i, 2, 1, 0] = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

        lat_perm[i, 0, 1, 2] = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
        lat_perm[i, 0, 2, 1] = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
        lat_perm[i, 1, 0, 2] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
        lat_perm[i, 1, 2, 0] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
        lat_perm[i, 2, 0, 1] = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
        lat_perm[i, 2, 1, 0] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
        # for p in possible_perms:
        #     lat_perm[i, p[0], p[1], p[2]] = perm_for_A1 @ lat_perm[0, p[0], p[1], p[2]]
        # perm_for_A2[i] = np.array([[1, 0, 0], [0, 0, 1], [0, 1, 0]])
    lat_perm = torch.tensor(lat_perm, dtype=torch.float32)
    perm_for_A1 = torch.tensor(perm_for_A1, dtype=torch.float32)
    perm_for_A2 = torch.tensor(perm_for_A2, dtype=torch.float32)
    return lat_perm.to(device), perm_for_A1.to(device), perm_for_A2.to(device)

def _project_to_space_group(batch,pos=None,cell=None):
    cf = CrystalFamily()
    cf.set_device(batch.pos.device)
    lat_perm, _, perm_for_A2 = get_latttice_permutations(device=batch.pos.device)
    if pos is not None:
        batch = batch.replace(pos=pos)
    else:
        batch = batch.replace(pos=torch.rand_like(batch.pos))
    if cell is not None:
        batch = batch.replace(cell=cell)
    else:
        batch = batch.replace(cell=torch.tensor(initialize_random_lattice().matrix).unsqueeze(0).float())

    wyckoff_batch = batch.wyckoff_bat.clone()
    idx, cum = 0, 0
    for len, num_atom in zip(batch.wyckoff_bat_len, batch.num_atoms):
        wyckoff_batch[idx : idx + len] = wyckoff_batch[idx : idx + len] + cum
        idx += len
        cum += num_atom

    # project latice
    conv_lat = torch.bmm(batch.prim_to_conv, batch.cell)
    conv_lat_vec = cf.m2v(cf.de_so3(conv_lat))
    conv_lat_vec_proj = cf.proj_k_to_spacegroup(conv_lat_vec, batch.space_groups)
    conv_lat_proj = cf.v2m(conv_lat_vec_proj)
    # print("conv_lat_proj") # The same
    # print(conv_lat_proj)

    rank = torch.argsort(-torch.norm(conv_lat_proj, dim=-1), dim=-1)
    idx = torch.cat([batch.space_groups.unsqueeze(-1), rank], dim=-1)
    # print(rank)
    perm = lat_perm[idx[:, 0], idx[:, 1], idx[:, 2], idx[:, 3]]
    # perm_A = perm_for_A2[batch.space_groups]    
    # perm = perm_for_A2[0].unsqueeze(0).repeat(batch.space_groups.shape[0], 1, 1)
    perm_conv_lat_proj = torch.bmm(torch.bmm(perm, conv_lat_proj), perm.transpose(-1, -2))
    # print("perm_conv_lat_proj")
    # print(perm_conv_lat_proj) # C, B, A
    
    
    rankA = torch.argsort(-torch.norm(perm_conv_lat_proj, dim=-1), dim=-1)
    idxA = torch.cat([batch.space_groups.unsqueeze(-1), rankA], dim=-1)
    permA = perm_for_A2[idxA[:, 0], idxA[:, 1], idxA[:, 2], idxA[:, 3]]
    # print("perm")
    # print(conv_lat_proj)
    # print(perm)
    # print(rank)
    # print("perm_conv_lat_proj")
    # print(perm_conv_lat_proj)
    # print(rankA)
    # print("permA")
    # print(permA)
    
    permA_conv_lat_proj = torch.bmm(torch.bmm(permA, perm_conv_lat_proj), permA.transpose(-1, -2))
    # print("permA_conv_lat_proj")
    # print(permA_conv_lat_proj)
    
    # Aligning the lattice vectors as the size of the lattice vectors
    # perm_conv_lat_proj = torch.bmm(
    #     torch.bmm(perm_A, perm_conv_lat_proj), perm_A.transpose(-1, -2)
    # )

    # perm_conv_lat_proj = conv_lat_proj

    # prim_lat_proj = torch.bmm(torch.bmm(torch.bmm(permA,batch.conv_to_prim),permA.transpose(-1,-2)), perm_conv_lat_proj)
    prim_lat_proj = torch.bmm(batch.conv_to_prim, permA_conv_lat_proj)
    
    pos_cart = torch.einsum("bi,bij->bj", batch.pos, batch.cell[batch.batch])
    # print(pos_cart)
    pos_cart = torch.einsum("bij,bj->bi", perm[batch.batch], pos_cart)
    pos_frac_conv = torch.einsum(
        "bi,bij->bj", pos_cart, torch.inverse(perm_conv_lat_proj)[batch.batch]
    )
    # print("inverse")
    pos_frac_conv = pos_frac_conv % 1.0
    pos_tran = torch.cat(
        [
            pos_frac_conv[wyckoff_batch],
            torch.ones(pos_frac_conv[wyckoff_batch].shape[0], 1, device=batch.pos.device),
        ],
        dim=1,
    )

    pos_frac_proj = (
        torch.einsum("bij,bj->bi", batch.wyckoff_ops, pos_tran).squeeze(-1)[:, :3] % 1.0
    )
    # trick = torch.tensor([0,1e-3,2e-3])
    trick = torch.tensor([0,0,0])
    # scatter(pos_frac_proj, batch.batch, dim=)
    # cannon_pos_frac_proj = pos_frac_proj[scatter_min((pos_frac_proj + trick).norm(dim=-1), wyckoff_batch, dim=0)[1][wyckoff_batch]]
    # pos_tran = torch.cat(
    #     [
    #         cannon_pos_frac_proj,
    #         torch.ones(cannon_pos_frac_proj.shape[0], 1, device=batch.pos.device),
    #     ]
    #     , dim=1
    # )
    # pos_frac_proj = (
    #     torch.einsum("bij,bj->bi", batch.wyckoff_ops, pos_tran).squeeze(-1)[:, :3] % 1.0
    # )
    
    # return pos_frac_proj, pos_frac_proj2
    pos_cart_proj = torch.einsum(
        "bi,bij->bj", pos_frac_proj, perm_conv_lat_proj[batch.batch[wyckoff_batch]]
    )
    prim_lat_proj = prim_lat_proj
    prim_lat_inv = torch.inverse(prim_lat_proj)
    pos_cart_proj = torch.einsum(
        "bij,bj->bi", permA[batch.batch[wyckoff_batch]], pos_cart_proj
    )
    pos_prim_frac_proj_all = torch.einsum(
        "bi,bij->bj", pos_cart_proj, prim_lat_inv[batch.batch[wyckoff_batch]]
    )

    ## Get prim idx
    for i in range(10):
        random_pos_frac_conv = torch.rand_like(pos_frac_conv).to(pos_frac_conv.device)
        random_pos_tran = torch.cat(
            [
                random_pos_frac_conv[wyckoff_batch],
                torch.ones(
                    random_pos_frac_conv[wyckoff_batch].shape[0],
                    1,
                    device=pos_frac_conv.device,
                ),
            ],
            dim=1,
        )
        random_pos_frac_proj = (
            torch.einsum("bij,bj->bi", batch.wyckoff_ops, random_pos_tran).squeeze(-1)[
                :, :3
            ]
            % 1.0
        ) % 1.0
        random_pos_cart_proj = torch.einsum(
            "bi,bij->bj",
            random_pos_frac_proj,
            perm_conv_lat_proj[batch.batch[wyckoff_batch]],
        )
        random_pos_cart_proj = torch.einsum(
        "bij,bj->bi", permA[batch.batch[wyckoff_batch]], random_pos_cart_proj
        )

        random_fracs = torch.einsum(
            "bi,bij->bj",
            random_pos_cart_proj,
            prim_lat_inv[batch.batch[wyckoff_batch]],
        )
        random_fracs = random_fracs % 1.0
        random_fracs_diff = random_fracs.unsqueeze(1) - random_fracs.unsqueeze(0)
        random_fracs_diff = random_fracs_diff - torch.round(random_fracs_diff)
        EPSILON = 5e-4
        random_fracs_diff_is_zero = torch.all(
            torch.isclose(
                random_fracs_diff,
                torch.zeros_like(random_fracs_diff),
                rtol=EPSILON,
                atol=EPSILON,
            ),
            dim=-1,
        )
        random_fracs_idx = random_fracs_diff_is_zero & (
            wyckoff_batch.unsqueeze(0) == wyckoff_batch.unsqueeze(1)
        )
        random_fracs_idx = ~(random_fracs_idx.triu(diagonal=1).any(dim=0))
        assert random_fracs_idx.shape[0] == pos_prim_frac_proj_all.shape[0]
        pos_prim_frac_proj = pos_prim_frac_proj_all[random_fracs_idx]
        if pos_prim_frac_proj.shape[0] == batch.pos.shape[0]:
            frac_batch = random_fracs_idx
            break
    pos_prim_frac_proj = pos_prim_frac_proj_all[frac_batch]


    return (pos_prim_frac_proj % 1.0) %1.0, prim_lat_proj, pos_frac_proj % 1.0, perm_conv_lat_proj

In [17]:
space_group_input = np.arange(1, 231).tolist()
info_path = "/home/holywater2/crystal_gen/mattergen/_my_scripts/space_group_info/prim/mp_20_val"

dataset = WyckoffDataset.from_space_group_list(
    space_group_infos_path=info_path,
    num_samples=1000,
    space_groups=39
)

In [5]:
pos_test = torch.rand_like(dataset[0].pos)
cell_test = torch.tensor(initialize_random_lattice().matrix).unsqueeze(0).float()

In [20]:
torch.tensor([0.3983, 0.2500, 0.0422]).norm()

tensor(0.4721)

In [22]:
torch.tensor([0.6017, 0.2500, 0.0422],).norm()

tensor(0.6529)

In [ ]:
torch.tensor([0.3983, 0.2500, 0.0422]).norm()

In [19]:
n=0
while(n < 100):
    n += 1
    pos_test = torch.rand_like(dataset[991].pos)
    cell_test = torch.tensor(initialize_random_lattice().matrix).unsqueeze(0).float()
    a,b,c,d = _project_to_space_group(collate([dataset[991]]), pos=pos_test, cell=cell_test)
    x,y,z,w = _project_to_space_group(collate([dataset[991]]),a,b)
    print(a)
    print(x)
    print(b)
    print(y)
    print(c)
    print(z)
    if not torch.allclose(a,x,rtol=1e-3) or not torch.allclose(b,y,rtol=1e-3) or not torch.allclose(c,z,rtol=1e-3) or not torch.allclose(d,w,rtol=1e-3):
        print("Fail")

        break

perm
tensor([[[7.7082, 0.0000, 0.0000],
         [0.0000, 8.2345, 0.0000],
         [0.0000, 0.0000, 6.1628]]])
tensor([[[0., 0., 1.],
         [0., 1., 0.],
         [1., 0., 0.]]])
tensor([[1, 0, 2]])
perm_conv_lat_proj
tensor([[[6.1628, 0.0000, 0.0000],
         [0.0000, 8.2345, 0.0000],
         [0.0000, 0.0000, 7.7082]]])
tensor([[1, 2, 0]])
permA
tensor([[[0., 0., 1.],
         [0., 1., 0.],
         [1., 0., 0.]]])
permA_conv_lat_proj
tensor([[[7.7082, 0.0000, 0.0000],
         [0.0000, 8.2345, 0.0000],
         [0.0000, 0.0000, 6.1628]]])
perm
tensor([[[7.7082, 0.0000, 0.0000],
         [0.0000, 8.2345, 0.0000],
         [0.0000, 0.0000, 6.1628]]])
tensor([[[0., 0., 1.],
         [0., 1., 0.],
         [1., 0., 0.]]])
tensor([[1, 0, 2]])
perm_conv_lat_proj
tensor([[[6.1628, 0.0000, 0.0000],
         [0.0000, 8.2345, 0.0000],
         [0.0000, 0.0000, 7.7082]]])
tensor([[1, 2, 0]])
permA
tensor([[[0., 0., 1.],
         [0., 1., 0.],
         [1., 0., 0.]]])
permA_conv_lat_proj
t

In [18]:

res = []
success = []
failed = []
error = []
consistency = []
inconsistency_data = {
}

torch.set_num_threads(24)
for idx, data in tqdm(enumerate(dataset)):
    data_batch = collate([data])
    _success = False
    for i in range(3):
        if not _success:
            coords, lattice, pos_frac_proj, conv_lat_proj = _project_to_space_group(data_batch)
            structure = Structure(lattice=lattice, coords=coords, species=data_batch.species)
            try:
                spga = SpacegroupAnalyzer(structure)
            except:
                print(f"Idx: {idx} failed, spg {data.space_groups}")
                continue
            _success = True
    if not _success:
        error.append(idx)
        failed.append(idx)
        continue
    
    coords2, lattice2, pos_frac_proj2, conv_lat_proj2 = _project_to_space_group(data_batch, coords, lattice)
    # coords2, lattice2, pos_frac_proj2, conv_lat_proj2 = _project_to_space_group(data_batch, coords2, lattice2)
    if torch.allclose(coords, coords2, atol=1e-3):
        consistency.append(idx)
    else:
        print(f"Idx: {idx} failed, spg {data.space_groups} inconsistent")    
        inconsistency_data[idx]={
            "data": data,
            "coords": coords,
            "coords2": coords2,
            "lattice": lattice,
            "lattice2": lattice2,
            "pos_frac_proj": pos_frac_proj,
            "pos_frac_proj2": pos_frac_proj2,
            "conv_lat_proj": conv_lat_proj,
            "conv_lat_proj2": conv_lat_proj2,
        }
    sgn = spga.get_space_group_number()
    res.append(sgn)
    if sgn == data.space_groups.item():
        success.append(idx)
    else:
        failed.append(idx)
        print(f"Idx: {idx} failed, spg {data.space_groups} != {sgn}")
print("Len of success: ", len(success))
print("Len of failed: ", len(failed))
print("Len of error: ", len(error))
print("Len of consistency: ", len(consistency))

0it [00:00, ?it/s]

Idx: 0 failed, spg 39 inconsistent
Idx: 1 failed, spg 39 inconsistent
Idx: 2 failed, spg 39 inconsistent
Idx: 3 failed, spg 39 inconsistent
Idx: 4 failed, spg 39 inconsistent
Idx: 5 failed, spg 39 inconsistent
Idx: 6 failed, spg 39 inconsistent
Idx: 8 failed, spg 39 inconsistent
Idx: 10 failed, spg 39 inconsistent
Idx: 12 failed, spg 39 inconsistent
Idx: 13 failed, spg 39 inconsistent
Idx: 15 failed, spg 39 inconsistent
Idx: 18 failed, spg 39 inconsistent
Idx: 21 failed, spg 39 inconsistent
Idx: 22 failed, spg 39 inconsistent
Idx: 24 failed, spg 39 inconsistent
Idx: 26 failed, spg 39 inconsistent
Idx: 27 failed, spg 39 inconsistent
Idx: 28 failed, spg 39 inconsistent
Idx: 29 failed, spg 39 inconsistent
Idx: 30 failed, spg 39 inconsistent
Idx: 31 failed, spg 39 inconsistent
Idx: 34 failed, spg 39 inconsistent
Idx: 35 failed, spg 39 inconsistent
Idx: 36 failed, spg 39 inconsistent
Idx: 38 failed, spg 39 inconsistent
Idx: 39 failed, spg 39 inconsistent
Idx: 41 failed, spg 39 inconsistent


In [19]:
inconsistency_data[996]

{'data': ChemGraph(pos=[4, 3], anchors=[4], anchors_len=4, wyckoff_ops=[8, 4, 4], wyckoff_ops_pinv=[8, 3, 3], wyckoff_bat=[8], wyckoff_bat_len=8, space_groups=39, num_atoms=4, conv_to_prim=[1, 3, 3], prim_to_conv=[1, 3, 3], species=[4], uniques=[4], uniques_len=4, num_nodes=4, atomic_numbers=[4], cell=[1, 3, 3]),
 'coords': tensor([[0.8602, 0.3602, 0.1984],
         [0.3602, 0.8602, 0.8016],
         [0.4472, 0.9472, 0.1747],
         [0.9472, 0.4472, 0.8253]]),
 'coords2': tensor([[0.3602, 0.8602, 0.1984],
         [0.8602, 0.3602, 0.8016],
         [0.4472, 0.9472, 0.1747],
         [0.9472, 0.4472, 0.8253]]),
 'lattice': tensor([[[ 3.9991, -6.1928,  0.0000],
          [ 3.9991,  6.1928,  0.0000],
          [ 0.0000,  0.0000,  8.4035]]]),
 'lattice2': tensor([[[ 3.9991, -6.1928,  0.0000],
          [ 3.9991,  6.1928,  0.0000],
          [ 0.0000,  0.0000,  8.4035]]]),
 'pos_frac_proj': tensor([[0.1984, 0.2500, 0.1102],
         [0.8016, 0.7500, 0.1102],
         [0.1984, 0.7500, 0.61

In [35]:

res = []
success = []
failed = []
error = []
consistency = []
inconsistency_data = {
}

torch.set_num_threads(24)
for idx, data in tqdm(enumerate(dataset)):
    data_batch = collate([data])
    _success = False
    for i in range(3):
        if not _success:
            coords, lattice, pos_frac_proj, conv_lat_proj = _project_to_space_group(data_batch)
            structure = Structure(lattice=lattice, coords=coords, species=data_batch.species)
            try:
                spga = SpacegroupAnalyzer(structure)
            except:
                print(f"Idx: {idx} failed, spg {data.space_groups}")
                continue
            _success = True
    if not _success:
        error.append(idx)
        failed.append(idx)
        continue
    
    coords2, lattice2, pos_frac_proj2, conv_lat_proj2 = _project_to_space_group(data_batch, coords, lattice)
    coords2, lattice2, pos_frac_proj2, conv_lat_proj2 = _project_to_space_group(data_batch, coords2, lattice2)
    if torch.allclose(coords, coords2, atol=1e-3):
        consistency.append(idx)
    else:
        print(f"Idx: {idx} failed, spg {data.space_groups} inconsistent")    
        inconsistency_data[idx]={
            "data": data,
            "coords": coords,
            "coords2": coords2,
            "lattice": lattice,
            "lattice2": lattice2,
            "pos_frac_proj": pos_frac_proj,
            "pos_frac_proj2": pos_frac_proj2,
            "conv_lat_proj": conv_lat_proj,
            "conv_lat_proj2": conv_lat_proj2,
        }
    sgn = spga.get_space_group_number()
    res.append(sgn)
    if sgn == data.space_groups.item():
        success.append(idx)
    else:
        failed.append(idx)
        print(f"Idx: {idx} failed, spg {data.space_groups} != {sgn}")
print("Len of success: ", len(success))
print("Len of failed: ", len(failed))
print("Len of error: ", len(error))
print("Len of consistency: ", len(consistency))

0it [00:00, ?it/s]

Idx: 14 failed, spg 119 inconsistent
Idx: 23 failed, spg 204 inconsistent
Idx: 40 failed, spg 107 inconsistent
Idx: 43 failed, spg 139 inconsistent
Idx: 75 failed, spg 82 inconsistent
Idx: 76 failed, spg 71 inconsistent
Idx: 82 failed, spg 155 inconsistent
Idx: 93 failed, spg 12 inconsistent
Idx: 101 failed, spg 74 inconsistent
Idx: 128 failed, spg 139 inconsistent
Idx: 131 failed, spg 12 inconsistent
Idx: 139 failed, spg 44 inconsistent
Idx: 152 failed, spg 227 inconsistent
Idx: 165 failed, spg 74 inconsistent
Idx: 170 failed, spg 139 inconsistent
Idx: 192 failed, spg 71 inconsistent
Idx: 259 failed, spg 12 inconsistent
Idx: 281 failed, spg 44 inconsistent
Idx: 310 failed, spg 44 inconsistent
Idx: 366 failed, spg 44 inconsistent
Idx: 389 failed, spg 74 inconsistent
Idx: 411 failed, spg 166 inconsistent
Idx: 413 failed, spg 44 inconsistent
Idx: 416 failed, spg 72 inconsistent
Idx: 419 failed, spg 71 inconsistent
Idx: 428 failed, spg 227 inconsistent
Idx: 438 failed, spg 141 inconsisten

In [40]:
inconsistency_data[964]

{'data': ChemGraph(pos=[13, 3], anchors=[13], anchors_len=13, wyckoff_ops=[26, 4, 4], wyckoff_ops_pinv=[26, 3, 3], wyckoff_bat=[26], wyckoff_bat_len=26, space_groups=71, num_atoms=13, conv_to_prim=[1, 3, 3], prim_to_conv=[1, 3, 3], species=[13], uniques=[13], uniques_len=13, num_nodes=13, atomic_numbers=[13], cell=[1, 3, 3]),
 'coords': tensor([[0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.5000],
         [1.0000, 0.0000, 0.5000],
         [1.0000, 0.5000, 0.0000],
         [0.5000, 0.0000, 0.0000],
         [0.9351, 0.4351, 0.5000],
         [0.0649, 0.5649, 0.5000],
         [0.1055, 0.0000, 0.1055],
         [0.8945, 0.0000, 0.8945],
         [0.3456, 0.3456, 0.0000],
         [0.6544, 0.6544, 0.0000],
         [0.0892, 0.5000, 0.5892],
         [0.9108, 0.5000, 0.4108]]),
 'coords2': tensor([[0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.5000],
         [0.0000, 0.0000, 0.5000],
         [0.0000, 0.5000, 0.0000],
         [0.5000, 0.0000, 0.0000],
         [0.4351, 0

In [42]:
inconsistency_data[964]["data"].conv_to_prim

tensor([[[-0.5000,  0.5000,  0.5000],
         [ 0.5000, -0.5000,  0.5000],
         [ 0.5000,  0.5000, -0.5000]]])

In [39]:
inconsistency_data[964]['pos_frac_proj'] - inconsistency_data[964]['pos_frac_proj2']

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  5.0000e-01],
        [ 0.0000e+00,  0.0000e+00, -5.0000e-01],
        [ 0.0000e+00,  0.0000e+00, -5.0000e-01],
        [ 0.0000e+00,  0.0000e+00,  5.0000e-01],
        [ 0.0000e+00, -7.4506e-09,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -5.9605e-08],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00

In [329]:
_project_to_space_group(collate(inconsistency_data[988]['data']), inconsistency_data[988]['coords2'], inconsistency_data[988]['lattice2'])

KeyError: 988

In [52]:
def test(data):
    return _project_to_space_group(collate(data['data']), data['coords2'], data['lattice2'])

In [53]:
test(inconsistency_data[991])

(tensor([[0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.0000]]),
 tensor([[[ 5.6984, -3.4987,  0.0000],
          [ 5.6984,  3.4987,  0.0000],
          [ 0.0000,  0.0000,  7.5526]]]),
 tensor([[0.5000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.5000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.5000, 0.5000],
         [0.5000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.5000],
         [0.5000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.5000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.5000, 0.5000],
         [0.5000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.5000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.5000, 0.5000]]),
 tensor([[[ 7.5526,  0.0000,  0.0000],
          [ 0.0000, 11.3969,  0.0000],
          [ 0.0000,  0.0000,  6.9974]]]))

In [ ]:
tt = torch.zeros(4,3)
tt[0][0] = 0.5
tt[0][1] = 0.5
_project_to_space_group(collate(inconsistency_data[997]['data']), inconsistency_data[997]['coords'] + tt, inconsistency_data[997]['lattice2'])

(tensor([[0.0413, 0.9587, 0.0000],
         [0.4285, 0.5715, 0.5000],
         [0.9914, 0.1041, 0.5000],
         [0.8959, 0.0086, 0.5000]]),
 tensor([[[ 4.9253, -2.9612,  0.0000],
          [ 4.9253,  2.9612,  0.0000],
          [ 0.0000,  0.0000,  8.0759]]]),
 tensor([[0.0000, 0.0000, 0.9587],
         [0.0000, 0.5000, 0.4587],
         [0.5000, 0.0000, 0.5715],
         [0.5000, 0.5000, 0.0715],
         [0.5000, 0.5477, 0.5563],
         [0.5000, 0.4523, 0.5563],
         [0.5000, 0.0477, 0.0563],
         [0.5000, 0.9523, 0.0563]]),
 tensor([[[8.0759, 0.0000, 0.0000],
          [0.0000, 9.8507, 0.0000],
          [0.0000, 0.0000, 5.9224]]]))

In [342]:
inconsistency_data[997]['coords'] + tt

tensor([[0.2913, 0.2087, 0.0000],
        [0.9285, 0.0715, 0.5000],
        [0.9914, 0.1041, 0.5000],
        [0.8959, 0.0086, 0.5000]])

In [54]:
inconsistency_data[991]

{'data': ChemGraph(pos=[7, 3], anchors=[7], anchors_len=7, wyckoff_ops=[14, 4, 4], wyckoff_ops_pinv=[14, 3, 3], wyckoff_bat=[14], wyckoff_bat_len=14, space_groups=38, num_atoms=7, conv_to_prim=[1, 3, 3], prim_to_conv=[1, 3, 3], species=[7], uniques=[7], uniques_len=7, num_nodes=7, atomic_numbers=[7], cell=[1, 3, 3]),
 'coords': tensor([[0.3364, 0.3364, 0.5000],
         [0.0529, 0.0529, 0.0000],
         [0.0825, 0.0825, 0.5000],
         [0.2610, 0.2610, 0.5000],
         [0.3855, 0.3855, 0.0000],
         [0.0591, 0.0591, 0.5000],
         [0.4593, 0.4593, 0.0000]]),
 'coords2': tensor([[0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.0000]]),
 'lattice': tensor([[[ 5.6984, -3.4987,  0.0000],
          [ 5.6984,  3.4987,  0.0000],
          [ 0.0000,  0.0000,  7.5526]]]),
 'lattice2': tensor([[[ 5.6984, -3.

In [55]:
inconsistency_data[991]['data'].wyckoff_ops

tensor([[[0.0000, 0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.0000, 0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 1.0000, 0.5000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 1.0000, 0.5000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.0000, 0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 1.0000]],

        [[0.0000, 0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 0.0000, 0.5000],
         [0.0000, 0.0000, 1.0000, 0.5000],
 

In [294]:
inconsistency_data[999]['data'].wyckoff_bat

tensor([0, 0, 1, 1, 2, 2, 2, 2])

In [266]:
coords2

tensor([[0.4597, 0.6486, 0.0000],
        [0.3514, 0.5403, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.6080, 0.2668, 0.0000],
        [0.7332, 0.3920, 0.0000],
        [0.0000, 0.0000, 0.5000],
        [0.6685, 0.1513, 0.5000],
        [0.8487, 0.3315, 0.5000],
        [0.0000, 0.0000, 0.0000],
        [0.9174, 0.2636, 0.0000],
        [0.7364, 0.0826, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.4904, 0.4511, 0.0000],
        [0.5489, 0.5096, 0.0000],
        [0.6182, 0.0268, 0.5000],
        [0.9732, 0.3818, 0.5000],
        [0.9021, 0.4930, 0.5000],
        [0.5070, 0.0979, 0.5000],
        [0.3404, 0.1641, 0.5000],
        [0.8359, 0.6596, 0.5000]])

In [270]:
lattice, lattice2

(tensor([[[ 4.2064, -2.5449,  0.0000],
          [ 4.2064,  2.5449,  0.0000],
          [ 0.0000,  0.0000,  5.7486]]]),
 tensor([[[ 4.2064, -2.5449,  0.0000],
          [ 4.2064,  2.5449,  0.0000],
          [ 0.0000,  0.0000,  5.7486]]]))

In [267]:
coords

tensor([[0.4597, 0.6486, 0.0000],
        [0.6486, 0.4597, 0.0000],
        [0.2180, 0.2180, 0.0000],
        [0.6080, 0.2668, 0.0000],
        [0.2668, 0.6080, 0.0000],
        [0.3282, 0.3282, 0.5000],
        [0.6685, 0.1513, 0.5000],
        [0.1513, 0.6685, 0.5000],
        [0.0999, 0.0999, 0.0000],
        [0.9174, 0.2636, 0.0000],
        [0.2636, 0.9174, 0.0000],
        [0.2413, 0.2413, 0.0000],
        [0.4904, 0.4511, 0.0000],
        [0.4511, 0.4904, 0.0000],
        [0.6182, 0.0268, 0.5000],
        [0.0268, 0.6182, 0.5000],
        [0.9021, 0.4930, 0.5000],
        [0.4930, 0.9021, 0.5000],
        [0.3404, 0.1641, 0.5000],
        [0.1641, 0.3404, 0.5000]])

In [269]:
_project_to_space_group(data_batch, coords2, lattice2)

(tensor([[0.4597, 0.6486, 0.0000],
         [0.3514, 0.5403, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.6080, 0.2668, 0.0000],
         [0.7332, 0.3920, 0.0000],
         [0.0000, 0.0000, 0.5000],
         [0.6685, 0.1513, 0.5000],
         [0.8487, 0.3315, 0.5000],
         [0.0000, 0.0000, 0.0000],
         [0.9174, 0.2636, 0.0000],
         [0.7364, 0.0826, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.4904, 0.4511, 0.0000],
         [0.5489, 0.5096, 0.0000],
         [0.6182, 0.0268, 0.5000],
         [0.9732, 0.3818, 0.5000],
         [0.9021, 0.4930, 0.5000],
         [0.5070, 0.0979, 0.5000],
         [0.3404, 0.1641, 0.5000],
         [0.8359, 0.6596, 0.5000]]),
 tensor([[[ 4.2064, -2.5449,  0.0000],
          [ 4.2064,  2.5449,  0.0000],
          [ 0.0000,  0.0000,  5.7486]]]),
 tensor([[0.0000, 0.5541, 0.0945],
         [0.0000, 0.4459, 0.0945],
         [0.0000, 0.0541, 0.5945],
         [0.0000, 0.9459, 0.5945],
         [0.0000, 0.0000, 0.0000],
   

In [223]:
structure

Structure Summary
Lattice
    abc : 8.877142777163153 8.877142777163153 6.539999961853027
 angles : 90.0 90.0 109.19969093458923
 volume : 486.7097915451345
      A : 5.142381191253662 -7.235991954803467 0.0
      B : 5.142381191253662 7.235991954803467 0.0
      C : 0.0 0.0 6.539999961853027
    pbc : True True True
PeriodicSite: Ca (5.212, 6.928, 0.0) [0.02799, 0.9855, 0.0]
PeriodicSite: Ca (5.212, -6.928, 0.0) [0.9855, 0.02799, 0.0]
PeriodicSite: Ca (3.652, 2.156e-07, 0.0) [0.3551, 0.3551, 0.0]
PeriodicSite: Er (6.521, -3.215, 0.0) [0.8562, 0.4119, 0.0]
PeriodicSite: Er (6.521, 3.215, 0.0) [0.4119, 0.8562, 0.0]
PeriodicSite: Sn (0.599, 2.696e-08, 3.27) [0.05824, 0.05824, 0.5]
PeriodicSite: Sn (5.738, 1.795, 3.27) [0.4339, 0.6819, 0.5]
PeriodicSite: Sn (5.738, -1.795, 3.27) [0.6819, 0.4339, 0.5]
PeriodicSite: S (3.647, 2.156e-07, 0.0) [0.3546, 0.3546, 0.0]
PeriodicSite: S (6.651, -4.768, 0.0) [0.9762, 0.3173, 0.0]
PeriodicSite: S (6.651, 4.768, 0.0) [0.3173, 0.9762, 0.0]
PeriodicSite

In [224]:
spga.get_refined_structure()

Structure Summary
Lattice
    abc : 6.539999961853027 14.471983909606934 10.284762382507324
 angles : 90.0 90.0 90.0
 volume : 973.419583090269
      A : 6.539999961853027 0.0 0.0
      B : 0.0 14.471983909606934 0.0
      C : 0.0 0.0 10.284762382507324
    pbc : True True True
PeriodicSite: Ca (0.0, 6.928, 5.212) [0.0, 0.4788, 0.5067]
PeriodicSite: Ca (0.0, 7.543, 5.212) [0.0, 0.5212, 0.5067]
PeriodicSite: Ca (0.0, 0.0, 3.652) [0.0, 0.0, 0.3551]
PeriodicSite: Ca (0.0, 14.16, 0.06929) [0.0, 0.9788, 0.006737]
PeriodicSite: Ca (0.0, 0.3075, 0.06929) [0.0, 0.02125, 0.006737]
PeriodicSite: Ca (0.0, 7.236, 8.795) [0.0, 0.5, 0.8551]
PeriodicSite: Er (0.0, 11.26, 6.521) [0.0, 0.7779, 0.6341]
PeriodicSite: Er (0.0, 3.215, 6.521) [0.0, 0.2221, 0.6341]
PeriodicSite: Er (0.0, 4.021, 1.379) [0.0, 0.2779, 0.1341]
PeriodicSite: Er (0.0, 10.45, 1.379) [0.0, 0.7221, 0.1341]
PeriodicSite: Sn (3.27, 0.0, 0.599) [0.5, 0.0, 0.05824]
PeriodicSite: Sn (3.27, 1.795, 5.738) [0.5, 0.124, 0.5579]
PeriodicSite: 

In [225]:
conv = Structure(lattice=conv_lat_proj, coords=pos_frac_proj, species=data_batch.species[data_batch.wyckoff_bat])
SpacegroupAnalyzer(conv).get_space_group_number()

38

In [226]:
conv

Structure Summary
Lattice
    abc : 6.539999961853027 14.471983909606934 10.284762382507324
 angles : 90.0 90.0 90.0
 volume : 973.419583090269
      A : 6.539999961853027 0.0 0.0
      B : 0.0 14.471983909606934 0.0
      C : 0.0 0.0 10.284762382507324
    pbc : True True True
PeriodicSite: Ca (0.0, 6.928, 5.212) [0.0, 0.4788, 0.5067]
PeriodicSite: Ca (0.0, 7.543, 5.212) [0.0, 0.5212, 0.5067]
PeriodicSite: Ca (0.0, 14.16, 0.06929) [0.0, 0.9788, 0.006737]
PeriodicSite: Ca (0.0, 0.3075, 0.06929) [0.0, 0.02125, 0.006737]
PeriodicSite: Ca (0.0, 0.0, 3.652) [0.0, 0.0, 0.3551]
PeriodicSite: Ca (0.0, 7.236, 8.795) [0.0, 0.5, 0.8551]
PeriodicSite: Er (0.0, 4.021, 1.379) [0.0, 0.2779, 0.1341]
PeriodicSite: Er (0.0, 10.45, 1.379) [0.0, 0.7221, 0.1341]
PeriodicSite: Er (0.0, 11.26, 6.521) [0.0, 0.7779, 0.6341]
PeriodicSite: Er (0.0, 3.215, 6.521) [0.0, 0.2221, 0.6341]
PeriodicSite: Sn (3.27, 0.0, 0.599) [0.5, 0.0, 0.05824]
PeriodicSite: Sn (3.27, 7.236, 5.741) [0.5, 0.5, 0.5582]
PeriodicSite: Sn

In [227]:
SpacegroupAnalyzer(conv).get_primitive_standard_structure()

Structure Summary
Lattice
    abc : 8.877142777163153 8.877142777163153 6.539999961853027
 angles : 90.0 90.0 109.19969093458923
 volume : 486.7097915451345
      A : 5.142381191253661 -7.235991954803467 -1.281966860120857e-16
      B : 5.142381191253663 7.235991954803467 7.579567525985208e-16
      C : 0.0 0.0 6.539999961853027
    pbc : True True True
PeriodicSite: Ca (5.212, 6.928, 7.434e-16) [0.02799, 0.9855, 0.0]
PeriodicSite: Ca (5.212, -6.928, 6.54) [0.9855, 0.02799, 1.0]
PeriodicSite: Ca (3.652, -8.882e-16, 2.236e-16) [0.3551, 0.3551, 0.0]
PeriodicSite: Er (6.521, -3.215, 2.025e-16) [0.8562, 0.4119, 0.0]
PeriodicSite: Er (6.521, 3.215, 6.54) [0.4119, 0.8562, 1.0]
PeriodicSite: Sn (0.599, 0.0, 3.27) [0.05824, 0.05824, 0.5]
PeriodicSite: Sn (5.738, 1.795, 3.27) [0.4339, 0.6819, 0.5]
PeriodicSite: Sn (5.738, -1.795, 3.27) [0.6819, 0.4339, 0.5]
PeriodicSite: S (3.647, -8.882e-16, 2.233e-16) [0.3546, 0.3546, 2.465e-32]
PeriodicSite: S (6.651, -4.768, 1.153e-16) [0.9762, 0.3173, 0.0]